# Cost Raster Calculations and Optimal Path

In [10]:
import requests
import zipfile
import arcpy

## Pre-processing

In [ ]:
# Adding the three DEMs together to create one raster layer
arcpy.management.MosaicToNewRaster("wabasha_clip_dem;dem_1m_m_Clip_winona;olmstead_dem_clip", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb", "county_dems", 'PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]', "16_BIT_UNSIGNED", None, 1, "LAST", "FIRST")

## Slope DEM

In [ ]:
# Calculating slope for the study area DEMs
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb"):
    out_raster = arcpy.sa.Slope("county_dems", "PERCENT_RISE", 1, "PLANAR", "METER"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb\Slope_county1")

In [ ]:
# Rescaling by function for slope
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb", compression="NONE", pyramid="NONE", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb"):
    out_raster = arcpy.sa.RescaleByFunction("Slope_county1", "MSSMALL 0.5 0.5 0 # 1000 #", 1, 10); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb\Rescale_coun1")

## Land Use Data

In [ ]:
# Reclassifying the land use data based on the outlined preferences
arcpy.ddd.Reclassify("NLCD_2016_Land_Cover_Clip", "Value", "11 7;21 1;22 1;23 1;24 1;31 1;41 1;42 1;43 1;52 1;71 1;81 10;82 10;90 5;95 5", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb\Reclass_NLCD1", "DATA")

In [ ]:
# Weighted Sum (both layers weighted as 1)
out_raster = arcpy.ia.WeightedSum("Reclass_NLCD1 Value 1;Rescale_coun1 VALUE 1"); out_raster.save(r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb\Weighte_Recl1")

In [ ]:
# Finding the Optimal Connection
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb", workspace=r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb"):
    arcpy.sa.OptimalRegionConnections("Points_XYTableToPoint1", r"C:\Users\ecava\OneDrive\Documents\ArcGIS\Projects\Lab2_pt2\Lab2_pt2.gdb\Optimal_Connection", None, "Weighte_Recl1", None, "PLANAR", "GENERATE_CONNECTIONS")

## Alternate Routes

Changing weights/classifications